In [1]:
# -*- coding: utf-8 -*-
import sys
import os       
import pandas as pd
import datetime

os.chdir('D:/zxdf/Workspaces/PycharmProjects/flask_ml')
from app import config, Elasticsearch_Util
#from app.lib import nltk_text_analyze_lib as textAnalyzer
#from app.lib import sklearn_classification_lib as skclassification

starttime = datetime.datetime.now()
print ("当前的日期和时间是 %s" % starttime)

In [2]:
def getDataFromES(es_util, index, type, beginTime, endTime, fromnum, size, source_include):
    # 最大查询10000条数据
    query_data = {
        "from": fromnum,
        "size": size,
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "@timestamp": {
                                "from": beginTime,
                                "to": endTime
                            }
                        }
                    }
                ],
            }
        },
        "_source": {
            "includes": source_include,
            "excludes": []
        },
        "sort": [
            {
                "@timestamp": {
                    "order": "desc"
                }
            }
        ]
    }
    res = es_util.query(index, type, query_data)
    df = None
    records = []
    for doc in res['hits']['hits']:
        records.append(doc['_source'])
    if records:
        df = pd.DataFrame(records)
    return df


In [3]:
from sklearn.preprocessing import LabelEncoder
index = "1012-knowledge_training-*"
type = "knowledge_training"
beginTime = "2018-01-30T00:00:00.000+08:00"
endTime = "2018-01-31T00:00:00.000+08:00"
fromnum = 0
size = 10000
#1085  804:281
#4212 :	544+260=804 
#5016
tag = "isException"
logmsg = "logBody"
source_include = [tag, logmsg]

es_util = Elasticsearch_Util()
train_data = getDataFromES(es_util, index=index, type=type,
                           beginTime=beginTime, endTime=endTime,
                           fromnum=fromnum, size=size,
                           source_include=source_include)
feature_data_array = train_data[logmsg]
class_data_array = train_data[tag]
print len(feature_data_array)
print len(class_data_array)

# 将类别转化为数字标签
#le = LabelEncoder()
#train_labelValues = le.fit_transform(class_data_array)

In [31]:
train_data

,isException,logBody
0,1,org.springframework.web.util.pattern.PatternPa...
1,1,org.springframework.web.util.NestedServletExce...
2,1,org.springframework.web.socket.sockjs.SockJsTr...
3,1,org.springframework.web.socket.sockjs.SockJsMe...
4,1,org.springframework.web.util.NestedServletExce...
5,1,org.springframework.web.socket.server.Handshak...
6,1,org.springframework.web.socket.handler.Session...
7,1,org.springframework.web.servlet.NoHandlerFound...
8,1,org.springframework.web.servlet.ModelAndViewDe...
9,1,org.springframework.web.server.ServerWebInputE...


In [5]:
#训练数据日志体
f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\logbody.txt", "w")
for indexs in train_data.index:
    print >> f, "%s%s" %(train_data.loc[indexs].values[0:1],train_data.loc[indexs].values[1:2])
f.close()

In [6]:
import nltk
import string
from nltk.corpus import stopwords
#from nltk.stem.porter import *

#训练数据关键词提取列表
def tokenize(text):
    f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\keyword_logbody.txt", "a+")
    tokens = nltk.word_tokenize(text)
    tokens2=[]
    for w in tokens:
        if w[0]=='-':
            tokens2.append(w[1:])
        elif w[0]=="'":
            tokens2.append(w[1:])    
        else:
            tokens2.append(w)
    
    stopword_tokens = [i for i in tokens2 if i not in string.punctuation]
    stopword_tokens2 = [w for w in stopword_tokens if w not in stopwords.words('english')]
    
    stopwords_custom=["''", "``","||","'/","'/'",u"'0x0","'2","'=","'s",'**','***','***.***','**constraint.unique_wt**','**failed','**method','--',
                      '-1','-297991290629036654','-614','-6182496564283649260','-6787310117729693199','-8','-999999999',u'-e',
                      '..','...','.\\xxx\\xxx.txt','/*','/**','/c','/p','0','1','==','b','c','e','x','===',
                      'v'
                     ]
    #,'mapped'
    stopword_tokens3 = [w for w in stopword_tokens2 if w not in stopwords_custom]    
            
    print >> f, "%s" %(stopword_tokens3) 
    f.close()
    return stopword_tokens3
#test = ['-application',
#'-authenticated',
#'-bean',
#'-before',
#"'classpathresourceloader",
#"'lasstemplateloader",
#"'losing",
#"'clustermanager"
#            ]
#filtered4=[]
#for w in test:
#    if w[0]=="'":
#        filtered4.append(w[1:])
#    else:
#        filtered4.append(w)
#print filtered3   

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
#max_features=100
#min_df=1
#max_df=
#textAnalyzer.
#token_pattern_custom=r"(?u)\b\w\w+\b"
#token_pattern=token_pattern_custom, analyzer='word',
vectorizer=CountVectorizer(tokenizer=tokenize, stop_words='english', decode_error='ignore')
csr_mat = vectorizer.fit_transform(feature_data_array)
# 获取词袋模型中的所有词
wordlist = vectorizer.get_feature_names()  
# tf矩阵 元素a[i][j]表示j词在i类文本中的tf
countlist = csr_mat.toarray()

In [8]:
#词频统计
f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\tf.txt", "w")
#print >> f,"%s" %  "keyword[",wordlist[j],"] tf[",countlist[i][j],"]"
print >> f, vectorizer.vocabulary_
f.close()
f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\tf_feature.txt", "w")
print >> f, "%s" % (wordlist)
f.close()

In [9]:
#对应文档的关键词词频统计
f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\tf_all.txt", "w")
# 打印每类文本的tf词语词频，第一个for遍历所有文本，第二个for便利某一类文本下的词频
for i in range(len(countlist)):
    print >> f,"%s"%"---------------------------------------------------line",i,"---------------------------------------------------"
    for j in range(len(wordlist)):
            #if  tfidfwordlist[j]=='null':
            if  countlist[i][j]>0:
                print >> f,"%s" %  "keyword[",wordlist[j],"] tf[",countlist[i][j],"]"
f.close()

In [10]:
# 提取特征
#from sklearn.feature_extraction.text import TfidfVectorizer
#max_features=100,
#min_df=200,
#token_pattern_custom=r"(?u)\b\w\w+\b"
#token_pattern=token_pattern_custom, analyzer='word',
#tf_transformer = TfidfVectorizer(tokenizer=tokenize, stop_words='english', decode_error='ignore')
#feature_datas = tf_transformer.fit_transform(feature_data_array)

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer=TfidfTransformer()
feature_datas= tf_transformer.fit_transform(csr_mat)

# 获取词袋模型中的所有词
#tfidfwordlist =tf_transformer.get_feature_names()
# tf-idf矩阵 元素a[i][j]表示j词在i类文本中的tf-idf权重
weighttlist = feature_datas.toarray()

In [ ]:
#f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\tfidf_feature.txt", "w")
#print >> f, "%s" % (tfidfwordlist[:])
#f.close()

In [11]:
f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\tf_tfidf.txt", "w")
for i in range(len(weighttlist)):
    for j in range(len(wordlist)):
            if  wordlist[j]=='mapped' and countlist[i][j]>0:
                print >> f,"%s" % "line(",i,") tf[",countlist[i][j],"],tf-idf[", weighttlist[i][j],"]"
f.close()                

In [12]:
#获得倒序的词频统计列表（词总数，包含该词的文档数，词，所有出现改词的文档列表（文档id，词数））
f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\tf_tfidfs.txt", "w")
listtext=[]
for j in range(len(wordlist)):    
    lineidcountarray=[]
    count=0
    #    print >> f,"%s" % "line(",i,") tf[",countlist[i][j],"],tf-idf[", weighttlist[i][j],"]"
    for i in range(len(weighttlist)):
        if countlist[i][j]>0:
            count+=countlist[i][j]
            lineidcountmap=(i,countlist[i][j])
            lineidcountarray.append(lineidcountmap)
    #print >> f,"%s" % "total[",count,"]keyword[",wordlist[j],"]",lineidcountarray
    textcontent=(count,len(lineidcountarray),wordlist[j],lineidcountarray)
    listtext.append(textcontent)
listtext=sorted(listtext, reverse=True)
for text in listtext:
    print >> f, text
f.close()  

In [13]:
#所有对应文档的tf与tf-idf
f = open(r"E:\zxdf\ml\linkdata-log\dataset\testrecord\tf_tfidf_all.txt", "w")
for i in range(len(weighttlist)):
    print >> f,"%s"%"---------------------------------------------------line",i,"---------------------------------------------------"
    for j in range(len(wordlist)):
            #if  tfidfwordlist[j]=='null':
            if  countlist[i][j]>0:
                print >> f,"%s" %  "keyword[",wordlist[j],"] tf[",countlist[i][j],"],tf-idf[", weighttlist[i][j],"]"
f.close() 

In [14]:
#验证算法
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
def compareAlgorithm(X_train,Y_train,num_folds=10, seed=7, scoring='accuracy'):
    models = {}
    models['KNN'] = KNeighborsClassifier()
    models['CART'] = DecisionTreeClassifier()
    #models['SVM'] = SVC()
    #models['LR'] = LogisticRegression()
    # models['LDA'] = LinearDiscriminantAnalysis()
    # models['NB'] = GaussianNB()
    # 评估算法
    results = []
    for key in models:
        kfold = KFold(n_splits=num_folds, random_state=seed)
        cv_results = cross_val_score(models[key], X_train, Y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        print('%s: %f (%f)' %(key, cv_results.mean(), cv_results.std()))
    return models, results

In [30]:
#X_train=feature_datas
#Y_train=class_data_array
#num_folds=10
#seed=7
#scoring='accuracy'
compareAlgorithm(X_train=feature_datas, Y_train=class_data_array)

({'CART': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=None,
              splitter='best'),
  'KNN': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, n_jobs=1, n_neighbors=5, p=2,
             weights='uniform')},
 [array([ 0.02390438,  0.44422311,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ]),
  array([ 0.39442231,  0.69721116,  0.99800797,  0.87649402,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  0.96007984])])

In [16]:

#CART分类树算法使用基尼系数来代替信息增益比，基尼系数代表了模型的不纯度，基尼系数越小，则不纯度越低，特征越好。这和信息增益(比)是相反的。
#对于生成的决策树做预测的时候，假如测试集里的样本A落到了某个叶子节点，而节点里有多个训练样本。
#则对于A的类别预测采用的是这个叶子节点里概率最大的类别。

#criterion="gini",
#splitter="best",random
#max_depth=None,
#min_samples_split=2,
#min_samples_leaf=1,
#min_weight_fraction_leaf=0.,
#max_features=None,
#random_state=None,
#max_leaf_nodes=None,
#min_impurity_decrease=0.,
#min_impurity_split=None,
#class_weight=None,balanced
#presort=False

#,class_weight='balanced'
model = DecisionTreeClassifier()
model.fit(feature_datas, class_data_array)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [28]:
from sklearn import tree
from IPython.display import Image  
import pydotplus
os.environ["PATH"] += os.pathsep + r'D:/zxdf/software/Graphviz2.38/bin/'

#['target_names', 'data', 'target', 'DESCR', 'feature_names']
error_data=feature_datas
# 获取词袋模型中的所有词
#error_feature_names = tf_transformer.get_feature_names()
error_feature_names = wordlist
error_target=class_data_array
#error_target_names=list(set(class_data_array))
#error_target_names=['1','0']

dot_data = tree.export_graphviz(model, out_file=None, 
                         feature_names=error_feature_names,  
                         class_names=error_target,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png()) 

In [27]:
testline='-Mapped java.lang.StringIndexOutOfBoundsException: String index out of range: 0 '

test_datas = []
featureValue = testline.lower().strip()
test_datas.append(featureValue)
#test_feature_datas = tf_transformer.transform(test_datas)
test_vect_datas=vectorizer.transform(test_datas)
test_feature_datas= tf_transformer.transform(test_vect_datas)
model.predict(test_feature_datas)

array([u'0'], dtype=object)